# Inspect key merge identifiers
We will examine representative yearly files from the betting-odds and engineered match datasets to confirm available key columns for merge: winner, loser, tournament.

Steps:
1. Load one year (2022) from each folder.
2. Detect columns containing winner / loser / tourney tokens.
3. Preview those columns.

Keeping code minimal so we can iterate next.

In [32]:
import pandas as pd, re, pathlib

# Root folders (adjust if paths differ)
odds_dir = pathlib.Path(r"c:\poly_code\Cleaned_Data_Betting-odds")
eng_dir = pathlib.Path(r"c:\poly_code\Cleaned_engineered_Data_Atp_matches")

# Pick a common year (prefer 2022 if present)
YEAR = 2022

# Helper to select file containing year
def pick_file(folder: pathlib.Path, year: int):
    pat = re.compile(str(year))
    for f in sorted(folder.iterdir()):
        if f.is_file() and pat.search(f.name) and f.suffix in {'.xlsx', '.csv'} and not f.name.startswith('~$'):
            return f
    return None

odds_file = pick_file(odds_dir, YEAR)
eng_file = pick_file(eng_dir, YEAR)
print("Selected files:")
print("Odds:", odds_file)
print("Engineered:", eng_file)

# Load (Excel vs CSV auto-handled)
def load(path):
    if path is None: return None
    if path.suffix == '.csv':
        return pd.read_csv(path)
    return pd.read_excel(path)

odds_df = load(odds_file)
eng_df = load(eng_file)
print('\nShapes -> odds:', None if odds_df is None else odds_df.shape, ' engineered:', None if eng_df is None else eng_df.shape)

# Candidate column detectors
KEY_TOKENS = {
    'winner': ['winner'],
    'loser': ['loser', 'player2'],
    'tourney': ['tourney', 'event', 'tournament']
}

def find_cols(df, tokens):
    if df is None: return []
    cols_lower = {c.lower(): c for c in df.columns}
    found = set()
    for t in tokens:
        for lc, orig in cols_lower.items():
            if t in lc:
                found.add(orig)
    return sorted(found)

odds_keys = {k: find_cols(odds_df, v) for k, v in KEY_TOKENS.items()}
eng_keys = {k: find_cols(eng_df, v) for k, v in KEY_TOKENS.items()}

print('\nDetected key columns (odds):', odds_keys)
print('Detected key columns (engineered):', eng_keys)

# Preview first few rows for overlapping identifiers
common_preview_cols = sorted(set(sum(odds_keys.values(), [])) & set(sum(eng_keys.values(), [])))
print('\nCommon identifier columns:', common_preview_cols)
if common_preview_cols and odds_df is not None and eng_df is not None:
    print('\nOdds sample:')
    print(odds_df[common_preview_cols].head(3))
    print('\nEngineered sample:')
    print(eng_df[common_preview_cols].head(3))
else:
    print('No common columns detected yet; may need name normalization next.')

Selected files:
Odds: c:\poly_code\Cleaned_Data_Betting-odds\cleaned_atp_2022.xlsx
Engineered: c:\poly_code\Cleaned_engineered_Data_Atp_matches\merged_matches_2022.xlsx

Shapes -> odds: (2547, 17)  engineered: (2556, 33)

Detected key columns (odds): {'winner': ['Winner', 'prob_winner'], 'loser': ['Loser', 'prob_loser'], 'tourney': ['Tournament']}
Detected key columns (engineered): {'winner': ['winner_age', 'winner_ht', 'winner_ioc', 'winner_name', 'winner_seed'], 'loser': ['loser_age', 'loser_ht', 'loser_ioc', 'loser_name', 'loser_seed'], 'tourney': ['tourney_id', 'tourney_level', 'tourney_name']}

Common identifier columns: []
No common columns detected yet; may need name normalization next.


# Normalize player names to a common key
We will convert both datasets' winner/loser names to the format: `Last F` (last name + space + first initial, no period).
This will help establish merge keys before tackling tournaments.

In [33]:
import math
import pandas as pd
import re

PARTICLES = {"de", "del", "de la", "van", "von", "da", "di", "dos", "du", "la", "le"}

# Basic cleaner
def _clean(s: str) -> str:
    s = re.sub(r"\s+", " ", s.strip())
    s = s.replace(" ", " ")  # non-breaking space
    return s

# Detect already-normalized: "Last F" or "Last F." patterns
def _is_last_initial(s: str) -> bool:
    parts = s.split()
    return len(parts) == 2 and (len(parts[1]) == 1 or (len(parts[1]) == 2 and parts[1].endswith('.')))

# Main normalization to "Last F"
def normalize_name(x):
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return None
    s = _clean(str(x))
    if not s:
        return None

    # If already like "Last F" or "Last F." -> ensure single space and drop period
    if _is_last_initial(s):
        last, ini = s.split()
        ini = ini[0]
        return f"{last} {ini.upper()}"

    # Handle comma: "Last, First Middle"
    if "," in s:
        last, rest = s.split(",", 1)
        last = _clean(last)
        rest = _clean(rest)
        first = rest.split()[0] if rest else ""
        return f"{last} {first[:1].upper()}" if first else last

    # No comma: try common patterns like "First Middle Last" or particles
    tokens = s.split()
    if len(tokens) == 1:
        return tokens[0]

    # Join two-token trailing surname if a known particle precedes the last token
    if len(tokens) >= 3 and tokens[-2].lower() in PARTICLES:
        last = tokens[-2] + " " + tokens[-1]
        first = tokens[0]
    else:
        first = tokens[0]
        last = tokens[-1]

    return f"{last} {first[:1].upper()}"

# Determine name columns in each df
odds_w, odds_l = None, None
for c in (odds_df.columns if isinstance(odds_df, pd.DataFrame) else []):
    cl = c.lower()
    if cl == 'winner' and odds_w is None:
        odds_w = c
    if cl == 'loser' and odds_l is None:
        odds_l = c

eng_w = 'winner_name' if isinstance(eng_df, pd.DataFrame) and 'winner_name' in eng_df.columns else None
eng_l = 'loser_name' if isinstance(eng_df, pd.DataFrame) and 'loser_name' in eng_df.columns else None

# Create normalized keys (only if columns exist)
if isinstance(odds_df, pd.DataFrame) and odds_w and odds_l:
    odds_df['winner_key'] = odds_df[odds_w].map(normalize_name)
    odds_df['loser_key'] = odds_df[odds_l].map(normalize_name)

if isinstance(eng_df, pd.DataFrame) and eng_w and eng_l:
    eng_df['winner_key'] = eng_df[eng_w].map(normalize_name)
    eng_df['loser_key'] = eng_df[eng_l].map(normalize_name)

# Preview
print("Odds name preview:")
if isinstance(odds_df, pd.DataFrame) and odds_w and odds_l:
    print(odds_df[[odds_w, 'winner_key', odds_l, 'loser_key']].head(5))
else:
    print("(name columns not found in odds_df)")

print("\nEngineered name preview:")
if isinstance(eng_df, pd.DataFrame) and eng_w and eng_l:
    print(eng_df[[eng_w, 'winner_key', eng_l, 'loser_key']].head(5))
else:
    print("(name columns not found in eng_df)")

Odds name preview:
        Winner  winner_key               Loser   loser_key
0    Kwon S.W.      S.W. K         Nishioka Y.  Nishioka Y
1  Monteiro T.  Monteiro T         Altmaier D.  Altmaier D
2     Djere L.     Djere L  Carballes Baena R.        R. C
3   Johnson S.   Johnson S            Vukic A.     Vukic A
4    Moutet C.    Moutet C             Rune H.      Rune H

Engineered name preview:
             winner_name    winner_key             loser_name    loser_key
0  Felix Auger Aliassime   Aliassime F  Roberto Bautista Agut       Agut R
1       Denis Shapovalov  Shapovalov D    Pablo Carreno Busta      Busta P
2  Roberto Bautista Agut        Agut R         Hubert Hurkacz    Hurkacz H
3    Pablo Carreno Busta       Busta P          Jan Zielinski  Zielinski J
4        Daniil Medvedev    Medvedev D  Felix Auger Aliassime  Aliassime F


# Normalize tournament names to a common key
We will create `tourney_key` in both datasets by:
- Lowercasing and stripping punctuation/digits
- Removing generic terms (e.g., "Open", "Masters", "Cup", "ATP", sponsors)
- Returning the remaining tokens Title-Cased (e.g., "Cordoba Open" -> "Cordoba").
This keeps things simple and aligns names across sources.

In [50]:
import re
import unicodedata
import pandas as pd

# Stopwords to drop from tournament names (keep minimal & generic)
STOPWORDS = {
    'open','masters','cup','championship','championships','tournament','atp','wta',
    'presented','presentedby','presented-by','by','the','tennis','international','internationals',
    'series','classic','bank','western','southern','rolex','mutua','bnp','paribas','rakuten','bet','betway',
    'master','finals','final','event','tour','world','masters1000','1000','500','250'
}

ROMAN_RE = re.compile(r'^(?=[MDCLXVI])(M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3}))$', re.I)
PUNCT_DIGIT_RE = re.compile(r"[^A-Za-zÀ-ÖØ-öø-ÿ\s]")

# French Open and Roland Garros equivalence
FRENCH_EQUIV = {"french open", "roland garros"}


def strip_accents(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))


def normalize_tourney(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    s = str(x).strip()
    if not s:
        return None

    # lowercase, remove accents
    s = strip_accents(s.lower())
    # remove punctuation (keep spaces) and digits/symbols
    s = PUNCT_DIGIT_RE.sub(' ', s)
    s = re.sub(r"\d+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    # Special rule: French Open and Roland Garros are the same
    s_nosp = s.replace(' ', '')
    if s in FRENCH_EQUIV or s_nosp in {w.replace(' ', '') for w in FRENCH_EQUIV}:
        return 'Roland Garros'
    if 'french' in s and 'open' in s:
        return 'Roland Garros'
    if 'roland' in s and 'garros' in s:
        return 'Roland Garros'

    tokens = [t for t in s.split(' ') if t]
    keep = []
    for t in tokens:
        if t in STOPWORDS:
            continue
        if ROMAN_RE.match(t):
            continue
        keep.append(t)

    if not keep:
        # fallback: first token if everything stripped
        keep = tokens[:1]

    # Title-case the result for readability
    return ' '.join(w.capitalize() for w in keep)

# Determine tournament columns
odds_t = None
if isinstance(odds_df, pd.DataFrame):
    for c in odds_df.columns:
        if c.lower() == 'tournament':
            odds_t = c
            break

eng_t = 'tourney_name' if isinstance(eng_df, pd.DataFrame) and 'tourney_name' in eng_df.columns else None

# Apply normalization
if isinstance(odds_df, pd.DataFrame) and odds_t:
    odds_df['tourney_key'] = odds_df[odds_t].map(normalize_tourney)

if isinstance(eng_df, pd.DataFrame) and eng_t:
    eng_df['tourney_key'] = eng_df[eng_t].map(normalize_tourney)

# Preview examples
print('Odds tournament preview:')
if isinstance(odds_df, pd.DataFrame) and odds_t:
    print(odds_df[[odds_t, 'tourney_key']].drop_duplicates().head(10))
else:
    print('(tournament column not found in odds_df)')

print('\nEngineered tournament preview:')
if isinstance(eng_df, pd.DataFrame) and eng_t:
    print(eng_df[[eng_t, 'tourney_key']].drop_duplicates().head(10))
else:
    print('(tourney_name column not found in eng_df)')

Odds tournament preview:
                 Tournament    tourney_key
0    Brisbane International       Brisbane
29    Hong Kong Tennis Open      Hong Kong
56   Adelaide International       Adelaide
83              ASB Classic            Asb
108         Australian Open     Australian
231      Open Sud de France  Sud De France
257            Cordoba Open        Cordoba
283             Dallas Open         Dallas
307                 Open 13           Open
334          Argentina Open      Argentina

Engineered tournament preview:
        tourney_name  tourney_key
0           Brisbane     Brisbane
29        United Cup       United
54         Hong Kong    Hong Kong
81          Adelaide     Adelaide
108         Auckland     Auckland
133  Australian Open   Australian
256      Montpellier  Montpellier
282          Cordoba      Cordoba
308           Dallas       Dallas
332        Marseille    Marseille


# Merge datasets on normalized keys
We will now merge the two datasets using the unique features:
- winner_key
- loser_key
- tourney_key
This will create a combined DataFrame for further modeling and analysis.

In [51]:
# Perform the merge on winner_key, loser_key, tourney_key
merge_keys = ['winner_key', 'loser_key', 'tourney_key']

# Only merge if both DataFrames are present and have the keys
if all(isinstance(df, pd.DataFrame) for df in [odds_df, eng_df]):
    missing_cols = [k for k in merge_keys if k not in odds_df.columns or k not in eng_df.columns]
    if missing_cols:
        print(f"Missing merge columns: {missing_cols}")
        merged_df = None
    else:
        merged_df = pd.merge(
            eng_df, odds_df,
            on=merge_keys,
            how='inner',
            suffixes=('_eng', '_odds')
        )
        print(f"Merged shape: {merged_df.shape}")
        print(merged_df[merge_keys + [col for col in merged_df.columns if col not in merge_keys][:5]].head())
else:
    print("One or both DataFrames missing; cannot merge.")

Merged shape: (1022, 53)
    winner_key    loser_key tourney_key tourney_id tourney_name surface  draw_size tourney_level
0   Dimitrov G       Rune H    Brisbane  2024-0339     Brisbane    Hard         32             A
1       Rune H  Safiullin R    Brisbane  2024-0339     Brisbane    Hard         32             A
2   Dimitrov G   Thompson J    Brisbane  2024-0339     Brisbane    Hard         32             A
3       Rune H  Duckworth J    Brisbane  2024-0339     Brisbane    Hard         32             A
4  Safiullin R    Arnaldi M    Brisbane  2024-0339     Brisbane    Hard         32             A


# Clean Merged_By_Year files: drop repeated features
We'll remove repeated columns from each yearly merged file to reduce duplication before further processing.

In [53]:
import pathlib
import pandas as pd

merged_dir = pathlib.Path(r"c:\poly_code\Merged_By_Year")
output_dir = merged_dir / "cleaned"
output_dir.mkdir(parents=True, exist_ok=True)

# Columns to drop (case-insensitive match)
DROP_COLS = {
    'tourney_key', 'location', 'tournament',
    'court', 'surface', 'round', 'best of',
    'winner', 'loser', 'winner_key', 'loser_key'
}

removed_summary = []
for f in sorted(merged_dir.iterdir()):
    if not f.is_file() or f.name.startswith('~$') or f.suffix.lower() not in {'.xlsx', '.csv'}:
        continue

    # Load
    if f.suffix.lower() == '.csv':
        df = pd.read_csv(f)
    else:
        df = pd.read_excel(f)

    # Map lower-case to original cols
    lc_map = {str(c).strip().lower(): c for c in df.columns}
    to_drop_real = [lc_map[k] for k in DROP_COLS if k in lc_map]

    # Drop
    if to_drop_real:
        df = df.drop(columns=to_drop_real, errors='ignore')

    # Save to cleaned folder (avoid in-place write locks)
    out_path = output_dir / f.name
    if f.suffix.lower() == '.csv':
        df.to_csv(out_path, index=False)
    else:
        df.to_excel(out_path, index=False)

    removed_summary.append((f.name, to_drop_real, out_path))

# Print concise summary
for name, cols, out in removed_summary:
    print(f"{name}: removed {len(cols)} cols -> {cols} | saved: {out}")
print("Done.")

merged_matches_2016.xlsx: removed 2 cols -> ['surface', 'round'] | saved: c:\poly_code\Merged_By_Year\cleaned\merged_matches_2016.xlsx
merged_matches_2017.xlsx: removed 2 cols -> ['surface', 'round'] | saved: c:\poly_code\Merged_By_Year\cleaned\merged_matches_2017.xlsx
merged_matches_2018.xlsx: removed 2 cols -> ['surface', 'round'] | saved: c:\poly_code\Merged_By_Year\cleaned\merged_matches_2018.xlsx
merged_matches_2019.xlsx: removed 2 cols -> ['surface', 'round'] | saved: c:\poly_code\Merged_By_Year\cleaned\merged_matches_2019.xlsx
merged_matches_2021.xlsx: removed 2 cols -> ['surface', 'round'] | saved: c:\poly_code\Merged_By_Year\cleaned\merged_matches_2021.xlsx
merged_matches_2022.xlsx: removed 2 cols -> ['surface', 'round'] | saved: c:\poly_code\Merged_By_Year\cleaned\merged_matches_2022.xlsx
merged_matches_2023.xlsx: removed 11 cols -> ['tourney_key', 'Tournament', 'Court', 'Surface', 'Location', 'Best of', 'winner_key', 'Winner', 'Round', 'Loser', 'loser_key'] | saved: c:\poly_